In [6]:
import torch
from transformer_lens import HookedTransformer
from datasets import load_dataset

In [2]:
cfact = load_dataset("azhx/counterfact")

Generating test split: 100%|██████████| 2191/2191 [00:00<00:00, 137739.74 examples/s]


In [47]:
model = HookedTransformer.from_pretrained("Qwen/Qwen2-1.5B", default_padding_side="left")
model.tokenizer.padding_side = "left"

Loaded pretrained model Qwen/Qwen2-1.5B into HookedTransformer


In [3]:
cfact

DatasetDict({
    train: Dataset({
        features: ['case_id', 'pararel_idx', 'requested_rewrite', 'paraphrase_prompts', 'neighborhood_prompts', 'attribute_prompts', 'generation_prompts'],
        num_rows: 19728
    })
    test: Dataset({
        features: ['case_id', 'pararel_idx', 'requested_rewrite', 'paraphrase_prompts', 'neighborhood_prompts', 'attribute_prompts', 'generation_prompts'],
        num_rows: 2191
    })
})

In [4]:
cfact["train"]["requested_rewrite"]

[{'prompt': 'The mother tongue of {} is',
  'relation_id': 'P103',
  'subject': 'Danielle Darrieux',
  'target_new': {'id': 'Q1860', 'str': 'English'},
  'target_true': {'id': 'Q150', 'str': 'French'}},
 {'prompt': 'The official religion of {} is',
  'relation_id': 'P140',
  'subject': 'Edwin of Northumbria',
  'target_new': {'id': 'Q432', 'str': 'Islam'},
  'target_true': {'id': 'Q5043', 'str': 'Christianity'}},
 {'prompt': '{}, the',
  'relation_id': 'P1303',
  'subject': 'Toko Yasuda',
  'target_new': {'id': 'Q5994', 'str': 'piano'},
  'target_true': {'id': 'Q6607', 'str': 'guitar'}},
 {'prompt': '{}, which is located in',
  'relation_id': 'P17',
  'subject': 'Autonomous University of Madrid',
  'target_new': {'id': 'Q34', 'str': 'Sweden'},
  'target_true': {'id': 'Q29', 'str': 'Spain'}},
 {'prompt': 'What is the twin city of {}? It is',
  'relation_id': 'P190',
  'subject': 'Lyon',
  'target_new': {'id': 'Q1461', 'str': 'Manila'},
  'target_true': {'id': 'Q3820', 'str': 'Beirut'}},

In [68]:
def get_key_cov(model, wiki_dataset):
    ''' Estimate E[KK^T] using 100k samples from wiki_dataset. '''
    ...

def correctness_filter(model, dataset, verbose=False):
    ''' Filter out any examples that the model gets wrong. '''
    def get_correctness(model, examples):
        ''' Populate dataset with first token of correct answer and model answer '''
        true_string = [' ' + r['target_true']['str'] for r in examples['requested_rewrite']]
        question_string = [r['prompt'].format(r['subject']) for r in examples['requested_rewrite']]
        print(question_string)
        model.tokenizer.padding_side = "right"
        correct_tokens = list(model.tokenizer(true_string, return_tensors="pt", padding=True)["input_ids"][:, 0])
        model.tokenizer.padding_side = "left"
        question_tokens = model.tokenizer(question_string, return_tensors="pt", padding=True)["input_ids"]
        model_preds = list(torch.argmax(
            model(question_tokens),
            dim=-1
        )[:, 0])
        if verbose:
            print(f"String: {true_string[0]} tokenized as {correct_tokens[0]}, model predicted {model_preds[0]}")

        return {"correct_token": correct_tokens, "model_pred": model_preds}
    
    with torch.no_grad():
        dataset = dataset.map(
            lambda row: get_correctness(model, row),
            batched=True,
            batch_size=1_000
        )
    return dataset

def get_subject_representations(model, dataset):
    ''' Get the subject representations k* for each example in the dataset. '''
    ...

def get_object_representations(model, dataset):
    ''' Get the object representations v* for each example in the dataset. '''
    ...

def get_rome_edit(model, dataset, cov_dataset):
    ''' Implement the ROME edit function. 
        W_hat = W + A(C^-1 * k_star)^T, where:
        k_star = E[mlp_out(x_i + subject)], x_i is random prefix
        C = covariance matrix of keys
        A = (v_star - W * k_star) / (C^-1 * k_star)^T * k_star
        v_star optimizes log probability of outputting correct object and KL with original model
    '''
    k_star = get_subject_representations(model, dataset)
    v_star = get_object_representations(model, dataset)
    C = get_key_cov(model, cov_dataset)
    A = (v_star - model.W * k_star) / torch.matmul(torch.linalg.inv(C), k_star).T * k_star

    return torch.matmul(A, torch.linalg.inv(C).T)

In [67]:
model.tokenizer.decode([323])

' and'

In [69]:
cfact["train"] = correctness_filter(model, cfact["train"], verbose=True)

Map:   0%|          | 0/19728 [00:00<?, ? examples/s]

['The mother tongue of Danielle Darrieux is', 'The official religion of Edwin of Northumbria is', 'Toko Yasuda, the', 'Autonomous University of Madrid, which is located in', 'What is the twin city of Lyon? It is', 'The mother tongue of Thomas Joannes Stieltjes is', 'Anaal Nathrakh, that was created in', 'Apple A5 was created by', 'What is the twin city of Wellington? It is', 'Shree Pundalik, created in', 'BBC One, by', 'Andreas Ivanschitz professionally plays the sport', 'Michel Denisot spoke the language', 'The mother tongue of Go Hyeon-jeong is', 'Percy Snow, the', 'Saint Petersburg is a twin city of', 'The original language of The Icelandic Dream was', 'Robert William Muench is a', 'Inner Circle railway line can be found in', 'Argentine Football Association belongs to the organization of', 'The headquarter of Monell Chemical Senses Center is located in', 'Charles Alfred Pillsbury expired at', 'What does Heath Brothers play? They play', 'Platform Controller Hub is created by', 'Billy

Map:   5%|▌         | 1000/19728 [00:03<01:00, 310.32 examples/s]

String:  French tokenized as 8585, model predicted 323
['The language used by Timo Koivusalo is', 'What sport does Marián Hossa play? They play', 'Beersheba is a twin city of', 'Phocaea is in', 'The Long Winters was founded in', 'George Mihai Florescu plays in the position of', 'The Chi-Lites is represented by music label', 'Coomera River can be found in', 'Dziga Vertov passed away in', 'Svenn Stray worked in the city of', 'Shironamhin, that was formulated in', 'The original language of Le Magnifique was', 'Where is Cleveland Classic? It is located in', 'Calder Freeway, in', 'The location of Radical War is', 'Pawn Stars debuted on', 'Windows Server 2003 is a product of', 'Marcelin Pleynet is a native speaker of', 'Halchidhoma is located in', 'Michel Pablo is originally from', 'Mohammad Hatta writes in', 'The original language of Doida Por Ti was', 'The location of Regina Coeli is', 'The language of The Harvest Month was', 'Robbery Under Arms, that originated in', 'Chakrabongse Bhuvanad

Map:  10%|█         | 2000/19728 [00:06<00:59, 295.92 examples/s]

...

String:  Amsterdam tokenized as 37741, model predicted 323
['The profession of Beata Tyszkiewicz is', 'The Naked Vicar Show, that was formulated in', 'Amol Palekar is native to', 'Paul Desjardins originates from', 'Said Seyam belongs to the organization of', 'George Hardinge speaks the language', 'Mentors started in', 'What is the twin city of Istanbul? It is', 'Cliff Morgan, of', 'The language of Captain Khorshid was', 'Wolf Gang started in', 'Georg Apenes worked in', 'Grace Lynn Kung is originally from', 'In Kaskinen, they understand', 'Braden Holtby plays in the position of', 'The language of France Info is', 'Napoleon Chagnon specializes in', 'Herman Wilhelm Bissen, who has a citizenship from', 'The headquarters of Library House is in', 'Super Audio CD is created by', 'Alevi is follower of', 'The Reputation, that was started in', 'Brzydula was developed in', 'Stephen Sackur, of', 'James Watt worked in the city of', 'What sport does Robbie Keane play? They play', 'Tile Ridge is loca

Map:  96%|█████████▋| 19000/19728 [01:00<00:02, 316.88 examples/s]

String:  actor tokenized as 12089, model predicted 323
['Mount Cocks is a part of the continent of', 'Bobobobs was from', 'Saint Margaret of Scotland lost their life at', 'Brock Lesnar professionally plays the sport', 'Lanyon Place railway station is located in', 'Ellen Gleditsch, who is a citizen of', 'Byron White professionally plays the sport', 'Eduardo Paes has the position of', 'Mookencheril Cherian Joseph has a citizenship from', "Sydney Theatre Company's headquarters are in", 'Sultan Ahmed Mosque is in', 'Debdas, formulated in', 'Call Cobbs, Jr. performs', "Michele Giordano's position is", 'Brian Paddick found employment in', 'Charles Grafly died in the city of', 'What does Cold Comfort Farm play? They play', 'Michel Peissel writes in', 'IBM Personal Computer, produced by', 'Haymo of Halberstadt holds the title of', 'Edvard Hagerup Bull was employed in', 'Tebey holds a citizenship from', 'Scotia Bay is located in', 'Jazz Party performs', 'Jeffrey Archer found employment in', 'Ye

Map: 100%|██████████| 19728/19728 [01:02<00:00, 314.49 examples/s]

String:  Antarctica tokenized as 71687, model predicted 323


In [50]:
cfact["train"]["correct_token"][0]

43197

In [51]:
model.tokenizer(cfact["train"]["requested_rewrite"][0]["target_true"]["str"])

{'input_ids': [43197], 'attention_mask': [1]}

In [ ]:
# Usage: model.W_out[layer] += get_rome_edit(model)